In [1]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [2]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 3E12-C811

 Directory of c:\Users\RAJAB IMAM\Documents\Python Projects\Data Pipeline

05/24/2022  09:39 AM    <DIR>          .
05/24/2022  09:39 AM    <DIR>          ..
05/23/2022  12:32 PM             3,458 csv_datasource.py
11/29/2020  09:04 PM               425 Customer Contracts$.csv
11/29/2020  09:04 PM               275 Customer Demo.csv
11/29/2020  09:04 PM               247 Customer Engagements.csv
05/23/2022  01:27 PM               436 customer_contracts.csv
05/22/2022  07:52 PM               154 Pipfile
05/22/2022  07:53 PM            24,563 Pipfile.lock
01/08/2022  02:05 AM             1,326 USArrests.csv
               8 File(s)         30,884 bytes
               2 Dir(s)  99,587,137,536 bytes free


In [9]:
#find csv files from working directory, isolate only csv, 


    #get names of only csv files
csv_files = []
for file in os.listdir(os.getcwd()):
        if file.endswith(".csv"):
            csv_files.append(file)



In [10]:
#create a new directory
dataset_dir = 'raw_data'
  
    #make dataset folder to process csv files
try: 
      mkdir = 'mkdir {0}'.format(dataset_dir)
      os.system(mkdir)
except:
      pass

    #move csv files to dataset folder
for csv in csv_files:
      mv_file = "mv '{0}' {1}".format(csv, dataset_dir)
      os.system(mv_file)
print(mv_file)

mv 'USArrests.csv' raw_data


In [11]:
for csv in csv_files:
      mv_file = "mv '{0}' {1}".format(csv, dataset_dir)
      os.system(mv_file)
      print(mv_file)

mv 'Customer Contracts$.csv' raw_data
mv 'Customer Demo.csv' raw_data
mv 'Customer Engagements.csv' raw_data
mv 'customer_contracts.csv' raw_data
mv 'USArrests.csv' raw_data


In [12]:
def create_df(dataset_dir, csv_files):
  
    data_path = os.getcwd()+'/'+dataset_dir+'/'

    #loop through the files and create the dataframe
    df = {}
    for file in csv_files:
        try:
            df[file] = pd.read_csv(data_path+file)
        except UnicodeDecodeError:
            df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1") #if utf-8 encoding error
        print(file)
    
    return df

In [13]:
def clean_tbl_name(filename):
  
    #rename csv, force lower case, no spaces, no dashes
    clean_tbl_name = filename.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")
    
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])

    return tbl_name

In [14]:
def clean_colname(dataframe):
  
    #force column names to be lower case, no spaces, no dashes
    dataframe.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace(".","_").replace("$","").replace("%","") for x in dataframe.columns]
    
    #processing data
    replacements = {
        'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp'
    }

    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    return col_str, dataframe.columns

In [18]:
def upload_to_db(host, dbname, user, password, tbl_name, col_str, file, dataframe, dataframe_columns):
    host='localhost'
    dbname='bigdata'
    user='postgres'
    password='Reason@2010'
    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)
    #conn_string = "host='localhost' dbname='bigdata' user='postgres' password='Reason@2010'"
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened database successfully')
    
    #drop table with same name
    cursor.execute("drop table if exists %s;" % (tbl_name))

    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    print('{0} was created successfully'.format(tbl_name)) 
    
    #insert values to table

    #save df to csv
    dataframe.to_csv(file, header=dataframe_columns, index=False, encoding='utf-8')

    #open the csv file, save it as an object
    my_file = open(file)
    print('file opened in memory')
    
    #upload to db
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
    print('file copied to db')
    
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    cursor.close()
    print('table {0} imported to db completed'.format(tbl_name))

    return upload_to_db
